In [1]:
###################################################
# PROJECT: Rating Products & Sorting Reviews on Amazon
###################################################

###################################################
# Problem Statement
###################################################

# One of the major issues in e-commerce is the accurate calculation of post-sale product ratings.
# Solving this problem will enhance customer satisfaction for the e-commerce site, improve product visibility for sellers,
# and ensure a smooth shopping experience for buyers. Another problem is properly sorting product reviews.
# Misleading reviews that are highlighted can directly impact sales, leading to financial loss and customer attrition.
# Addressing these two key issues will help e-commerce sites and sellers boost their sales while ensuring customers
# have a seamless purchasing journey.

###################################################
# Dataset Story
###################################################

# This dataset contains Amazon product data, including product categories and various metadata.
# It includes user ratings and reviews for the most reviewed products in the Electronics category.

# Variables:
# reviewerID: User ID
# asin: Product ID
# reviewerName: User Name
# helpful: Helpful rating score
# reviewText: Review text
# overall: Product rating
# summary: Review summary
# unixReviewTime: Review time (Unix timestamp)
# reviewTime: Raw review time
# day_diff: Number of days since the review
# helpful_yes: Number of times the review was marked as helpful
# total_vote: Total number of votes for the review


In [2]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
###################################################
# TASK 1: Calculate the Average Rating Based on Recent Reviews and Compare It with the Existing Average Rating.
###################################################

# In the provided dataset, users have given ratings and written reviews for a product.
# The objective of this task is to assess the ratings by weighting them according to the date.
# The new average rating, calculated based on the weighted dates, needs to be compared with the existing average rating.

###################################################
# Step 1: Load the Dataset and Calculate the Product's Average Rating.
###################################################


In [4]:
df = pd.read_csv("/content/amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'day_diff',
       'helpful_yes', 'total_vote'],
      dtype='object')

In [6]:
df.shape

(4915, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [8]:
df.dtypes

,0
reviewerID,object
asin,object
reviewerName,object
helpful,object
reviewText,object
overall,float64
summary,object
unixReviewTime,int64
reviewTime,object
day_diff,int64


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.00000,4.58759,0.99685,1.00000,5.00000,5.00000,5.00000,5.00000
unixReviewTime,4915.00000,1379465001.66836,15818574.32275,1339200000.00000,1365897600.00000,1381276800.00000,1392163200.00000,1406073600.00000
day_diff,4915.00000,437.36704,209.43987,1.00000,281.00000,431.00000,601.00000,1064.00000
helpful_yes,4915.00000,1.31109,41.61916,0.00000,0.00000,0.00000,0.00000,1952.00000
total_vote,4915.00000,1.52146,44.12309,0.00000,0.00000,0.00000,0.00000,2020.00000


In [11]:
df["overall"].mean()

4.587589013224822

In [13]:
###################################################
# Adım 2: Tarihe Göre Ağırlıklı Puan Ortalamasını Hesaplayınız.
###################################################

for col in df.columns:
  if "Time" in col:
    df[col] = pd.to_datetime(df[col])

In [15]:
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [18]:
date = df["reviewTime"].max()
date

Timestamp('2014-12-07 00:00:00')

In [19]:
df["days"] = (date - df["reviewTime"]).dt.days

In [20]:
df['days_quantile'] = pd.qcut(df['days'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
df.loc[df['days_quantile'] == 'Q1', 'overall'].mean() * (30 / 100) + \
df.loc[df['days_quantile'] == 'Q2', 'overall'].mean() * (26 / 100) + \
df.loc[df['days_quantile'] == 'Q3', 'overall'].mean() * (24 / 100) + \
df.loc[df['days_quantile'] == 'Q4', 'overall'].mean() * (20 / 100)

4.600583941300071

In [21]:
df['day_diff_quantile'] = pd.qcut(df['day_diff'], q=4, labels=['Q1_diff', 'Q2_diff', 'Q3_diff', 'Q4_diff'])
# To observe which time zone it is in
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return df.loc[df['day_diff_quantile'] == 'Q1_diff', 'overall'].mean() * (w1 / 100) + \
           df.loc[df['day_diff_quantile'] == 'Q2_diff', 'overall'].mean() * (w2 / 100) + \
           df.loc[df['day_diff_quantile'] == 'Q3_diff', 'overall'].mean() * (w3 / 100) + \
           df.loc[df['day_diff_quantile'] == 'Q4_diff', 'overall'].mean() * (w4 / 100)


In [22]:
time_based_weighted_average(df)


4.595593165128118

In [23]:
quantiles = ['Q1_diff', 'Q2_diff', 'Q3_diff', 'Q4_diff']
for i in quantiles :
    print('mean of :', i, df.loc[df['day_diff_quantile'] == i, 'overall'].mean())

Q1_diff ortalaması : 4.6957928802588995
Q2_diff ortalaması : 4.636140637775961
Q3_diff ortalaması : 4.571661237785016
Q4_diff ortalaması : 4.4462540716612375


In [24]:
# Ratings given recently are higher. It can be said that the product is popular.

In [25]:
###################################################
# TASK 2: Determine 20 Reviews to be Displayed on the Product Detail Page.
###################################################

###################################################
# Step 1: Create the helpful_no Variable
###################################################
# Note:
# total_vote is the total number of up and down votes given to a review.
# up means helpful.
# The dataset does not have a helpful_no variable; it needs to be created from the existing variables.

df["helpful_no"] = (df["total_vote"] - df["helpful_yes"])
df["helpful_yes"].sum()
df["helpful_no"].sum()

1034

In [26]:
###################################################
# Step 2: Calculate and Add the score_pos_neg_diff, score_average_rating, and wilson_lower_bound Scores to the Data
###################################################

def score_up_down_diff(up, down):
    return up - down

In [27]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [28]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)
"""
Ya da up ve down adlı iki yeni değişken oluşturulup yapılabilir.
up = df["helpful_yes"]
down = df["helpful_no"]
"""

'\nYa da up ve down adlı iki yeni değişken oluşturulup yapılabilir.\nup = df["helpful_yes"]\ndown = df["helpful_no"]\n'

In [29]:
df['score_pos_neg_diff'] = df.apply(lambda x: score_up_down_diff(x['helpful_yes'],
                                                                 x['helpful_no']), axis=1)


In [30]:
df['score_average_rating'] = df.apply(lambda x: score_average_rating(x['helpful_yes'],
                                                                     x['helpful_no']), axis=1)


In [31]:
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'],
                                                                 x['helpful_no']), axis=1)


In [32]:
###################################################
# Step 3: Identify 20 Reviews and Interpret the Results.
###################################################

df.sort_values('wilson_lower_bound', ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,days_quantile,day_diff_quantile,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1970-01-01 00:00:01.367366400,2013-01-05,702,1952,2020,701,Q4,Q4_diff,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1970-01-01 00:00:01.348617600,2012-09-26,803,1428,1505,802,Q4,Q4_diff,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1970-01-01 00:00:01.375660800,2013-05-08,579,1568,1694,578,Q3,Q3_diff,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1970-01-01 00:00:01.346544000,2012-02-09,1033,422,495,1032,Q4,Q4_diff,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1970-01-01 00:00:01.394150400,2014-07-03,158,45,49,157,Q1,Q1_diff,4,41,0.91837,0.80811
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1970-01-01 00:00:01.393545600,2014-02-28,283,60,68,282,Q2,Q2_diff,8,52,0.88235,0.78465
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1970-01-01 00:00:01.350864000,2012-10-22,777,112,139,776,Q4,Q4_diff,27,85,0.80576,0.73214
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.00000,"Good buy for the money but wait, I had an issue!",1970-01-01 00:00:01.361923200,2013-02-27,649,22,25,648,Q4,Q4_diff,3,19,0.88000,0.70044
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1970-01-01 00:00:01.339200000,2012-09-06,823,51,65,822,Q4,Q4_diff,14,37,0.78462,0.67033
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1970-01-01 00:00:01.348272000,2012-09-22,807,82,109,806,Q4,Q4_diff,27,55,0.75229,0.66359
